## Load data into memory

In [130]:
from dotenv import load_dotenv
import sys
sys.path.append('..')
from src.repo.orm import OpenPool, PrayerRequestORM
from src.dto.prayerRequests import PrayerRequest
load_dotenv()
import os
import pandas as pd
pg_uri = os.environ.get('PRAYERS_PG_DATABASE_URL')
pool = OpenPool(pg_uri)

In [217]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
import numpy as np
from datetime import datetime, UTC
from sklearn.preprocessing import StandardScaler

class FeatureBuilder:
    def __init__(self):
        pass

    def build_preprocess(self, data: list[dict])->pd.DataFrame:
        today = datetime.now(UTC)
        for obj in data:
            obj['dayLength'] = (today - obj['created_at']).days
        return pd.DataFrame(data)

    def build_postprocess(self, paired_components: np.ndarray, use_day_feature = True, use_similarity_feature = True, has_predictive=True)->np.ndarray:
        processed = []
        for contact_group in paired_components:
            for pairs in contact_group:
                left_pair = pairs[0]
                right_pair = pairs[1]
                left_vector: np.ndarray = left_pair[0]
                right_vector: np.ndarray = right_pair[0]
                left_days = left_pair[1]
                right_days = right_pair[1]
                similarity = cosine(left_vector, right_vector)
                averaged_vector = (left_vector + right_vector) / 2.0

                extra_features = self.build_y_feature(left_pair, right_pair, has_predictive)
                if use_day_feature:
                    extra_features.insert(0, right_days)
                    extra_features.insert(0, left_days)
                if use_similarity_feature:
                    extra_features.insert(0, similarity)
                new_feature_set = np.concatenate([averaged_vector, extra_features])
                processed.append(new_feature_set)

        return np.array(processed)
    
    def build_postprocess2(self, paired_components: np.ndarray, use_day_feature = True, use_similarity_feature = True, has_predictive=True)->np.ndarray:
        processed = []
        for contact_group in paired_components:
            for pairs in contact_group:
                left_pair = pairs[0]
                right_pair = pairs[1]
                left_vector: np.ndarray = left_pair[0]
                right_vector: np.ndarray = right_pair[0]
                left_days = left_pair[1]
                right_days = right_pair[1]
                similarity = cosine(left_vector, right_vector)

                extra_features = self.build_y_feature(left_pair, right_pair, has_predictive)
                if use_day_feature:
                    extra_features.insert(0, right_days)
                    extra_features.insert(0, left_days)
                if use_similarity_feature:
                    extra_features.insert(0, similarity)
                new_feature_set = np.concatenate([left_vector, right_vector, extra_features])
                processed.append(new_feature_set)

        return np.array(processed)
    
    def build_y_feature(self, left_pair, right_pair, has_predictive:bool)->list:
        if not has_predictive:
            return []
        y = False
        if left_pair[2] == right_pair[2] and not np.isnan(left_pair[2]):
            y = True
        return [y]
                

class Preprocessor:
    def __init__(self):
        self.scaler = StandardScaler()
        self.columns = ["dayLength"]

    def fit_transform(self, data: list[dict]):
        self.fit(data)
        return self.transform(data)

    def fit(self, df: pd.DataFrame):
        self.scaler.fit(df[self.columns])

    def transform(self, df: pd.DataFrame)->np.ndarray:
        self.scaler.transform(df[self.columns])
        df["embedding"].to_numpy()
        # nparray = df.apply(lambda x: np.append(x["embedding"], x["dayLength"]), axis=1).to_numpy()
            
        return df[["embedding", "dayLength", "link_id", "prayer_request"]].to_numpy()


In [153]:
all_requests = []
with pool() as session:
    prayer_requests = session.query(PrayerRequestORM).all()
    for request in prayer_requests:
        if request.request == "":
            continue
        requestObject = {
            "prayer_request": request.request,
            "embedding": request.gte_base_embedding,
            "link_id": request.link_id,
            "created_at": request.created_at,
            "contact_id": request.contact_id
        }
        all_requests.append(requestObject)

df = pd.DataFrame(all_requests)
df.head(2)

,prayer_request,embedding,link_id,created_at,contact_id
0,Mom is thinking she wants to move to Tennessee...,"[-0.009760048, -0.005248226, 0.0009992049, -0....",182.0,2024-05-20 23:19:10.768122+00:00,29
1,Spanish ministry has been twice a month gather...,"[-0.014935532, -0.032861583, 0.012076679, 0.00...",175.0,2024-04-15 23:14:53.639172+00:00,28


In [218]:

contact_groups = {}
feature_builder = FeatureBuilder()
preprocessor = Preprocessor()
preprocess_features = feature_builder.build_preprocess(all_requests)
preprocessed = preprocessor.fit_transform(preprocess_features)

for i in range(len(preprocessed)):
    if all_requests[i]['contact_id'] not in contact_groups:
        contact_groups[all_requests[i]['contact_id']] = []
    contact_groups[all_requests[i]['contact_id']].append(preprocessed[i])

In [219]:
from itertools import combinations


paired_components = []
for index in contact_groups:
    group = contact_groups[index]
    paired_components.append(list(combinations(group, 2)))

In [220]:
final_feature_set = feature_builder.build_postprocess(paired_components, use_day_feature=True, use_similarity_feature=True)
# final_feature_set = feature_builder.build_postprocess2(paired_components, use_day_feature=False, use_similarity_feature=False)
len(final_feature_set)


25650

In [221]:
from sklearn.model_selection import train_test_split

y = final_feature_set[:,-1]
X = final_feature_set[:, :-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [238]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=42, learning_rate=0.01)
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
model.score(X_test, y_test)

0.9907894736842106


0.9884990253411307

In [223]:
# Scores seem suspicious, and so do the weights. Seems like a classic case of overfitting
# Would like to experiment in production
model.feature_importances_.std()

10.335164574506615

In [250]:
import sys
sys.path.append('..')
from src.repo.orm import ContactORM
from sentence_transformers import SentenceTransformer
from datetime import datetime, UTC
import pandas as pd
contactNameToId = {}
with pool() as session:
    contacts = session.query(ContactORM).all()
    for contact in contacts:
        contactNameToId[contact.name] = contact.id

testUserContactID = contactNameToId['Test User']

gte_transformer = SentenceTransformer("thenlper/gte-base")

def ImitateUser_FindRelated(contactID : int, request : str)->np.ndarray:
    embeddings = gte_transformer.encode(request, convert_to_numpy=True)
    created_at = datetime.now(UTC)
    link_id = np.nan
    imitate_request = {
        "prayer_request": request,
        "embedding": embeddings,
        "link_id": link_id,
        "created_at": created_at,
        "contact_id": contactID
    }
    preprocess_features = feature_builder.build_preprocess([imitate_request])
    preprocessed = preprocessor.transform(preprocess_features)
    paired_components = []
    for element in contact_groups[contactID]:
        paired_components.append((preprocessed[0], element))
    final_features = feature_builder.build_postprocess([paired_components], has_predictive=False)
    predictions = model.predict(final_features)
    indexes = np.where(predictions == 1)
    if len(indexes) == 0:
        return []
    return np.array(paired_components)[indexes][:, :,3]

related_requests = ImitateUser_FindRelated(testUserContactID, "Family is continuing to improve")
related_requests

array([['Family is continuing to improve',
        'An update on my family, they are getting better']], dtype=object)